# Lab 11: Feature Engineering & Cross-Validation

In this lab, you will gain practice with using the scikit learn library to do some feature engineering and how to use cross-validation to produce a model with the least error for new data.

In [ ]:
# Convenience function for instructor use only.
# Call this function to force refresh the okpy tests.
def refresh():
    import sys
    keys = [k for k in sys.modules.keys() if 'ok_tests' in k]
    for k in keys:
        del sys.modules[k]
    global ok
    ok = Notebook('lab11.ok')

In [ ]:
# Run this cell to set up your notebook
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
sns.set_context("talk")

from IPython.display import display, Latex, Markdown
from client.api.notebook import Notebook
ok = Notebook('lab11.ok')

### Introduction

For this lab, we will be using a toy dataset to predict the house prices in Boston with data provided by the `sklearn.datasets` package.

Run the following cell to load the data. This will return a dictionary object which includes keys for:
    - `data` : the data to learn
    - `target` : the response vector
    - `feature_names`: the column names
    - `DESCR` : a full description of the data

In [ ]:
from sklearn.datasets import load_boston

boston_data = load_boston()
print(boston_data.keys())

A look at the `DESCR` attribute tells us the data contains these features:

    1. CRIM      per capita crime rate by town
    2. ZN        proportion of residential land zoned for lots over 
                 25,000 sq.ft.
    3. INDUS     proportion of non-retail business acres per town
    4. CHAS      Charles River dummy variable (= 1 if tract bounds 
                 river; 0 otherwise)
    5. NOX       nitric oxides concentration (parts per 10 million)
    6. RM        average number of rooms per dwelling
    7. AGE       proportion of owner-occupied units built prior to 1940
    8. DIS       weighted distances to five Boston employment centres
    9. RAD       index of accessibility to radial highways
    10. TAX      full-value property-tax rate per 10,000 USD
    11. PTRATIO  pupil-teacher ratio by town
    12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks 
                 by town
    13. LSTAT    % lower status of the population
    
Let's now convert this data into a pandas DataFrame. 

In [ ]:
boston = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
boston.head()

### Question 1

Let's train some data! Before we can do this however, we need to split our data into a training set and validation set. These hold out points will be used to choose our best model. Additionally, remember that the response vector (housing prices) lives separate of the data in the `target` attribute.

Use the [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to split out one third of the training data for validation. Call the resulting datasets `X_train`, `X_test`, `Y_train`, `Y_test`.

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(42)

X = boston
Y = pd.Series(boston_data.target)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
assert X_train.shape == (339, 13)
assert X_test.shape == (167, 13)
assert Y_train.shape == (339, )
assert Y_test.shape == (167, )

### Question 2

As a warmup, let's regress a line to this data using `sklearn`. We've imported `sklearn.linear_model` as lm, so you can use that instead of typing out the whole module name. Running the cell should create a scatter plot for our predictions vs the true prices.

In [ ]:
import sklearn.linear_model as lm

linear_clf = lm.LinearRegression()

# Fit your classifier
linear_clf.fit(X_train, Y_train)

# Predict points from our test set
Y_pred = linear_clf.predict(X_test)

# Plot predicted vs true prices
plt.scatter(Y_test, Y_pred, alpha=0.5)
plt.xlabel("Prices")
plt.ylabel("Predicted prices")
plt.title("Prices vs Predicted prices")

### Question 3

As shown from the scatter plot, our model is not perfect. Ideally, we would see a line of slope 1 from the points if our model was 100% accurate.

Now let's also compute our mean squared error. Fill out the function below and compute the MSE for our predictions on both the training data `X_train` and the test set `X_test`.

In [ ]:
def mse(predicted_y, actual_y):
    return np.mean((predicted_y - actual_y) ** 2)

train_error = mse(linear_clf.predict(X_train), Y_train)
test_error = mse(Y_pred, Y_test)

train_error, test_error

In [ ]:
assert np.allclose((train_error, test_error), (22.98349, 20.74714))

### Question 4

We have just executed a linear regression on our data. Can we get this predictor to be any better though? The following sections will go through cross-validation and some feature engineering in order to try and produce the best model with the least error.

Let's start by seeing what is the **single** best feature for predicting boston house prices. For each feature in the given features list, fit a Linear Regression model to the training set where just that feature is selected for and check its accuracy on the validation set. Use the `mse` function you defined above.

In [ ]:
features = boston_data.feature_names

# Your code to find the single best feature
def compute_error(feature):
    train = X_train[[feature]]
    test = X_test[[feature]]
    linear_clf.fit(train, Y_train)
    return mse(linear_clf.predict(test), Y_test)

errors = [compute_error(feature) for feature in features]
best_feature_idx = np.argmin(errors)
best_feature, best_error = features[best_feature_idx], errors[best_feature_idx]

best_feature, best_error

In [ ]:
assert best_feature == 'LSTAT'
assert np.abs(best_error - 38.410075117662345) <= 1e-3

## K-Folds Cross Validation

We've so far been working with the generalized method of cross-validation where we split our data into a test and train set. Now let's try k-folds cross-validation to select the best subset of features for our model. Recall the approach looks something like:

<img src="cv.png" width=500px>

### Question 5

For the sake of this lab, we have a provided a list of feature subsets and will try to find out which subset produces the best model predictor. In future assignments (Project 2?!?!), you will be given the full feature set and use EDA and other techniques you've learned in this class to select your set of best features.

Here are the necessary steps to find your best linear predictor:

1. Use the [`KFold.split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) function to get 4 splits on your training data. Note that `split` returns the indices of the data for that split.
2. For each split, select out the rows and columns based on the split indices and features.
3. Compute the MSE from your prediction.
4. Based on the set with that gave the smallest MSE, choose your best set of features.


In [ ]:
from sklearn.model_selection import KFold
feature_sets = [['TAX', 'INDUS', 'CRIM'], ['RM', 'LSTAT', 'PTRATIO'], ['RM', 'B', 'NOX'], ['TAX', 'LSTAT', 'DIS']]

kf = KFold(n_splits=4)
splits = kf.split(X_train)

def compute_error(train_idx, valid_idx, features):
    split_X_train, split_X_valid = X_train.iloc[train_idx][features], X_train.iloc[valid_idx][features]
    split_Y_train, split_Y_valid = Y_train.iloc[train_idx], Y_train.iloc[valid_idx]
    
    linear_clf.fit(split_X_train, split_Y_train)
    return mse(linear_clf.predict(split_X_valid), split_Y_valid)

errors = []
for i in range(len(feature_sets)):
    train_idx, valid_idx = next(splits)
    feat = feature_sets[i]
    error = compute_error(train_idx, valid_idx, feat)
    print('{}, error: {}'.format(feat, error))
    errors.append(error)
    
best_err_idx = np.argmin(errors)

best_feature_set = feature_sets[best_err_idx]
best_feature_set

In [ ]:
assert best_feature_set == ['RM', 'LSTAT', 'PTRATIO']

### Question 6
Finally, fit a linear classifier using your best feature set and predict housing prices for your original test set. Compute the final MSE.

In [ ]:

# Fit your classifier
linear_clf.fit(X_train[best_feature_set], Y_train)

# Predict points from our test set
final_mse = mse(linear_clf.predict(X_test[best_feature_set]), Y_test)
final_mse

In [ ]:
assert np.abs(final_mse - 24.98372270187488) < 1e-3

Nice! You've used kfolds cross-validation to fit an accurate linear regression model to the dataset.

In the future, you'd probably want to use something like [`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) to automatically perform cross-validation, but it's instructive to do it yourself at least once.

In [ ]:
# Log into OkPy.
# You might need to change this to ok.auth(force=True) if you get an error
ok.auth(force=False)

## Submission

Run the cell below to run all the OkPy tests at once:

In [ ]:
import os
print("Running all tests...")
_ = ok.grade_all()

Now, run the cell below to submit your assignment to OkPy. The autograder should email you shortly with your autograded score. The autograder will only run once every 30 minutes.

**If you're failing tests on the autograder but pass them locally**, you should simulate the autograder by doing the following:

1. In the top menu, click Kernel -> Restart and Run all.
2. Run the cell above to run each OkPy test.

**You must make sure that you pass all the tests when running steps 1 and 2 in order.** If you are still failing autograder tests, you should double check your results.

In [ ]:
_ = ok.submit()